# Punto 2 Búsqueda binaria usando índice invertido (BSII)
## Integrantes
* Juan Esteban Arboleda
* Luccas Rojas

In [29]:
import os
import sys
import unicodedata
import sklearn
import pandas as pd

# Ruta que se debe cambiar segun la ubicacion del archivo, al descomprimir el zip no deberia afectar
#  '../data/docs-raw-texts/wes2015.d001.naf'

documents_path = '../data/docs-raw-texts'

def load_documents(documents_folder_path):
    dataframe = []
    for filename in os.listdir(documents_folder_path):
        document = pd.read_xml(os.path.join(documents_folder_path, filename))['raw']
        dataframe.append(document)
        #NFKC
    return dataframe

data = load_documents(documents_path)
len(data)

331